<a href="https://colab.research.google.com/github/manikantha4576/Fmml__IIIT__H_Assignmentss/blob/main/Module_01_Lab_02_MLPractice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Machine learning terms and metrics

FMML Module 1, Lab 2<br>


 In this lab, we will show a part of the ML pipeline by extracting features, training and testing

In [1]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn import datasets
# set randomseed
rng = np.random.default_rng(seed=42)

In this lab, we will use the California Housing dataset. There are 20640 samples, each with 8 attributes like income of the block, age of the houses per district etc. The task is to predict the cost of the houses per district.

Let us download and examine the dataset.

In [2]:
 dataset =  datasets.fetch_california_housing()
 # print(dataset.DESCR)  # uncomment this if you want to know more about this dataset
 # print(dataset.keys())  # if you want to know what else is there in this dataset
 dataset.target = dataset.target.astype(np.int) # so that we can classify
 print(dataset.data.shape)
 print(dataset.target.shape)

(20640, 8)
(20640,)


<ipython-input-2-60ae2e9a125e>:4: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dataset.target = dataset.target.astype(np.int) # so that we can classify


Here is a function for calculating the 1-nearest neighbours

In [3]:
def NN1(traindata, trainlabel, query):
  diff  = traindata - query  # find the difference between features. Numpy automatically takes care of the size here
  sq = diff*diff # square the differences
  dist = sq.sum(1) # add up the squares
  label = trainlabel[np.argmin(dist)] # our predicted label is the label of the training data which has the least distance from the query
  return label

def NN(traindata, trainlabel, testdata):
  # we will run nearest neighbour for each sample in the test data
  # and collect the predicted classes in an array using list comprehension
  predlabel = np.array([NN1(traindata, trainlabel, i) for i in testdata])
  return predlabel

We will also define a 'random classifier', which randomly allots labels to each sample

In [4]:
def RandomClassifier(traindata, trainlabel, testdata):
  # in reality, we don't need these arguments

  classes = np.unique(trainlabel)
  rints = rng.integers(low=0, high=len(classes), size=len(testdata))
  predlabel = classes[rints]
  return predlabel

Let us define a metric 'Accuracy' to see how good our learning algorithm is. Accuracy is the ratio of the number of correctly classified samples to the total number of samples. The higher the accuracy, the better the algorithm.

In [5]:
def Accuracy(gtlabel, predlabel):
  assert len(gtlabel)==len(predlabel), "Length of the groundtruth labels and predicted labels should be the same"
  correct = (gtlabel==predlabel).sum() # count the number of times the groundtruth label is equal to the predicted label.
  return correct/len(gtlabel)

Let us make a function to split the dataset with the desired probability.

In [6]:
def split(data, label, percent):
  # generate a random number for each sample
  rnd = rng.random(len(label))
  split1 = rnd<percent
  split2 = rnd>=percent
  split1data = data[split1,:]
  split1label = label[split1]
  split2data = data[split2,:]
  split2label = label[split2]
  return split1data, split1label, split2data, split2label

We will reserve 20% of our dataset as the test set. We will not change this portion throughout our experiments

In [7]:
testdata, testlabel, alltraindata, alltrainlabel = split(dataset.data, dataset.target, 20/100)
print('Number of test samples = ', len(testlabel))
print('Number of other samples = ', len(alltrainlabel))
print('Percent of test data = ', len(testlabel)*100/len(dataset.target),'%')

Number of test samples =  4144
Number of other samples =  16496
Percent of test data =  20.07751937984496 %


## Experiments with splits

Let us reserve some of our train data as a validation set

In [8]:
traindata, trainlabel, valdata, vallabel = split(alltraindata, alltrainlabel, 75/100)

What is the accuracy of our classifiers on the train dataset?

In [9]:
trainpred = NN(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Train accuracy using nearest neighbour is ", trainAccuracy)

trainpred = RandomClassifier(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Train accuracy using random classifier is ", trainAccuracy)

Train accuracy using nearest neighbour is  1.0
Train accuracy using random classifier is  0.164375808538163


For nearest neighbour, the train accuracy is always 1. The accuracy of the random classifier is close to 1/(number of classes) which is 0.1666 in our case.

Let us predict the labels for our validation set and get the accuracy

In [10]:
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using nearest neighbour is ", valAccuracy)

valpred = RandomClassifier(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using random classifier is ", valAccuracy)

Validation accuracy using nearest neighbour is  0.34108527131782945
Validation accuracy using random classifier is  0.1688468992248062


Validation accuracy of nearest neighbour is considerably less than its train accuracy while the validation accuracy of random classifier is the same. However, the validation accuracy of nearest neighbour is twice that of the random classifier.

Now let us try another random split and check the validation accuracy

In [11]:
traindata, trainlabel, valdata, vallabel = split(alltraindata, alltrainlabel, 75/100)
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy of nearest neighbour is ", valAccuracy)

Validation accuracy of nearest neighbour is  0.34048257372654156


You can run the above cell multiple times to try with different random splits.
We notice that the accuracy is different for each run, but close together.

Now let us compare it with the accuracy we get on the test dataset.

In [12]:
testpred = NN(alltraindata, alltrainlabel, testdata)
testAccuracy = Accuracy(testlabel, testpred)
print('Test accuracy is ', testAccuracy)

Test accuracy is  0.34917953667953666


### Try it out for yourself and answer:
1. How is the accuracy of the validation set affected if we increase the percentage of validation set? What happens when we reduce it?
2. How does the size of the train and validation set affect how well we can predict the accuracy on the test set using the validation set?
3. What do you think is a good percentage to reserve for the validation set so that thest two factors are balanced?

Answer for both nearest neighbour and random classifier. You can note down the values for your experiments and plot a graph using  <a href=https://matplotlib.org/stable/gallery/lines_bars_and_markers/step_demo.html#sphx-glr-gallery-lines-bars-and-markers-step-demo-py>plt.plot<href>. Check also for extreme values for splits, like 99.9% or 0.1%

## Multiple Splits

One way to get more accurate estimates for the test accuracy is by using <b>crossvalidation</b>. Here, we will try a simple version, where we do multiple train/val splits and take the average of validation accuracies as the test accuracy estimation. Here is a function for doing this. Note that this function will take a long time to execute.

In [13]:
# you can use this function for random classifier also
def AverageAccuracy(alldata, alllabel, splitpercent, iterations, classifier=NN):
  accuracy = 0
  for ii in range(iterations):
    traindata, trainlabel, valdata, vallabel = split(alldata, alllabel, splitpercent)
    valpred = classifier(traindata, trainlabel, valdata)
    accuracy += Accuracy(vallabel, valpred)
  return accuracy/iterations # average of all accuracies

In [14]:
print('Average validation accuracy is ', AverageAccuracy(alltraindata, alltrainlabel, 75/100, 10, classifier=NN))
testpred = NN(alltraindata, alltrainlabel, testdata)
print('test accuracy is ',Accuracy(testlabel, testpred) )

Average validation accuracy is  0.33584635395170215
test accuracy is  0.34917953667953666


This is a very simple way of doing cross-validation. There are many well-known algorithms for cross-validation, like k-fold cross-validation, leave-one-out etc. This will be covered in detail in a later module. For more information about cross-validation, check <a href=https://en.wikipedia.org/wiki/Cross-validation_(statistics)>Cross-validatioin (Wikipedia)</a>

### Questions
1. Does averaging the validation accuracy across multiple splits give more consistent results?
2. Does it give more accurate estimate of test accuracy?
3. What is the effect of the number of iterations on the estimate? Do we get a better estimate with higher iterations?
4. Consider the results you got for the previous questions. Can we deal with a very small train dataset or validation dataset by increasing the iterations?


**ANSWER 1**

Yes, averaging the validation accuracy across multiple splits, typically in the context of cross-validation, can indeed provide more consistent and reliable results when evaluating the performance of a machine learning model. This approach is commonly used to obtain a more robust estimate of a model's generalization performance.

Here's why averaging validation accuracy across multiple splits can be beneficial:

Reduced Variance: When you split your dataset into multiple subsets (folds) and train/validate your model on each fold, you get multiple estimates of your model's performance. Averaging these estimates reduces the variance in your evaluation, making it less sensitive to the specific random data splits.

Better Generalization Estimate: By testing your model on different subsets of the data, you obtain a more comprehensive assessment of how well it generalizes to unseen data. This helps ensure that your model's performance is not just good for one specific data partition but is indicative of its overall performance.

Reduced Risk of Overfitting: Averaging across multiple splits can help identify if your model is overfitting or underfitting. Consistent performance across folds suggests better generalization, while significant variability may indicate overfitting or instability.

Improved Confidence Intervals: You can compute confidence intervals or standard deviations on the averaged metrics, which provide insights into the level of confidence you can have in your model's performance estimate.

Better Hyperparameter Tuning: Cross-validation can be used in hyperparameter tuning to find the best set of hyperparameters that lead to the most consistent and robust model performance.

Common cross-validation techniques include k-fold cross-validation (where you split the data into k subsets and train/validate the model k times), stratified cross-validation (ensuring that class distributions are similar in each fold), and leave-one-out cross-validation (a special case where each data point serves as its own validation set).

However, it's essential to keep in mind that cross-validation can be computationally expensive, especially for large datasets and complex models. It may not always be necessary, and sometimes a simple train-validation split is sufficient. The choice of cross-validation method and the number of splits (k) depends on your specific dataset and modeling goals.







**ANSWER2**

Averaging the validation accuracy across multiple splits, such as in cross-validation, does not give a more accurate estimate of test accuracy. Instead, it provides a more accurate estimate of how well your model is likely to perform on unseen data, which is a measure of its generalization performance.

Here's the distinction:

Validation Accuracy: This is used to assess how well your model is performing on a portion of your dataset that it has not seen during training. The purpose is to tune hyperparameters, select the best model, and understand its behavior. It doesn't provide an accurate estimate of how your model will perform on completely new, unseen data.

Test Accuracy: This is a measure of your model's performance on completely new and unseen data, which is typically separate from both the training and validation data. It is used to evaluate how well your model generalizes to real-world, out-of-sample data.

Cross-validation, by averaging validation accuracy across multiple splits, provides a better estimate of how well your model generalizes to new data compared to a single validation split. However, it is still an estimate and not a direct measure of test accuracy on completely unseen data.



**answer3**

The number of iterations (or epochs) in the context of training a machine learning model can have an impact on the estimate of the model's performance, but it doesn't necessarily lead to a better estimate with higher iterations. The relationship between the number of iterations and the estimate of model performance is more nuanced and depends on various factors:

Underfitting and Overfitting: The number of iterations is closely related to the model's capacity to fit the training data. If you have too few iterations, the model may underfit the data, leading to poor performance. If you have too many iterations, the model can overfit the data, performing well on the training data but poorly on new, unseen data. Therefore, the right number of iterations strikes a balance between underfitting and overfitting.

Early Stopping: To find the optimal number of iterations, it's common practice to use techniques like early stopping. Early stopping monitors the model's performance on a validation set during training and stops training when performance starts to degrade. This prevents overfitting and can lead to a better estimate of how well the model will generalize.

Training Data Size: The number of iterations can also depend on the size of your training data. With a smaller dataset, you may need fewer iterations, while a larger dataset may require more iterations to converge.

Complexity of the Model: More complex models may require more iterations to learn the underlying patterns in the data. Simpler models might converge quickly with fewer iterations.

Stochastic Variability: Some training algorithms have inherent stochasticity (e.g., stochastic gradient descent), and the estimate of model performance can vary from one training run to another, even with the same hyperparameters and dataset. Averaging results over multiple runs can help reduce this variability.

Regularization: The use of regularization techniques, such as dropout or L1/L2 regularization, can affect the optimal number of iterations. Regularization helps control overfitting and can allow you to train for more iterations without overfitting.



**answer4**

Increasing the number of iterations can help to some extent when dealing with a very small training dataset, but it is not a guaranteed solution, and there are limitations to what it can achieve. Here are some considerations:

Advantages of Increasing Iterations with a Small Training Dataset:

Model Learning: With more iterations, the model has more opportunities to learn from the limited training data. This can help the model capture more complex patterns and potentially improve its performance.

Avoiding Underfitting: In some cases, if the initial number of iterations is too low, the model may underfit the training data. Increasing iterations can mitigate underfitting to some extent.

Limitations and Considerations:

Overfitting Risk: Increasing the number of iterations significantly with a very small training dataset can lead to overfitting. The model may start fitting noise in the data, resulting in poor generalization to new, unseen data. It's crucial to monitor the model's performance on a validation set and use techniques like early stopping to prevent overfitting.

Data Quality: If your training dataset is very small and not representative of the population you're interested in, increasing iterations may not help much. The model can only learn what is present in the data. If the data is noisy, unrepresentative, or contains biases, more iterations won't fundamentally improve the model's performance.

Model Complexity: The complexity of the model matters. Very complex models may require a large amount of data to generalize effectively. Increasing iterations alone may not compensate for a lack of data.

Regularization: If your dataset is small, it's often beneficial to use regularization techniques, such as dropout or L1/L2 regularization, to help control overfitting. These techniques can complement increased iterations by stabilizing the learning process.
